In [1]:
import numpy as np
import pandas as pd
import os
import torch
from utils.csv_to_pd import *

In [2]:
df = read_dir_csv()

location_ori = list(df["LocationCode"]) 
df[:1]

,LocationCode,DateTime,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW)
0,10,2024-03-01 17:14:06.000,0.0,1017.48,15.59,94.3,652.92,0.12


In [3]:
df = mean_10min(df)
df[:1]


,DateTime,LocationCode,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW),hour
0,2024-03-01 17:10:00,10.0,0.124286,1017.49,15.712857,93.771429,652.797143,0.115714,17


In [4]:
from sklearn.preprocessing import StandardScaler
import category_encoders as ce

encoder = ce.LeaveOneOutEncoder(cols=["LocationCode", "hour"], sigma = 0.05)
encoder.fit(df, df['Power(mW)'])
df = encoder.transform(df)

df[:1]


,DateTime,LocationCode,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW),hour
0,2024-03-01 17:10:00,171.555961,0.124286,1017.49,15.712857,93.771429,652.797143,0.115714,2.273539


In [5]:
# 指定要標準化的欄位
columns_to_standardize = ['WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', "LocationCode", "hour"]

# 初始化 StandardScaler
scaler = StandardScaler()

# 對指定欄位進行標準化
df[columns_to_standardize] = scaler.fit_transform(df[columns_to_standardize])

df[:1]

,DateTime,LocationCode,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW),hour
0,2024-03-01 17:10:00,-0.757742,-0.284717,0.741951,-1.640724,0.974532,-0.721713,0.115714,-1.015733


In [6]:
import random
def split_data_random(data_label_list, train_ratio=0.95):

    # 創建索引列表並隨機打亂
    indices = list(range(len(data_label_list)))
    random.shuffle(indices)

    # 計算分割點
    train_size = int(len(data_label_list) * train_ratio)

    # 分配數據
    train_indices = indices[:train_size]
    valid_indices = indices[train_size:]

    # 根據索引分割數據
    train_data_label_list = [data_label_list[i] for i in train_indices]
    valid_data_label_list = [data_label_list[i] for i in valid_indices]


    return train_data_label_list, valid_data_label_list

In [7]:
data_label_list, _ = spilt_data_with_datetime(df, location_ori)

train_data_label_list, valid_data_label_list = split_data_random(data_label_list)

train_data, train_label, train_length = sort_by_length(train_data_label_list)
valid_data, valid_label, valid_length = sort_by_length(valid_data_label_list)

In [8]:
from torch.nn.utils.rnn import pad_sequence

def padding(data_list, label_list, length, batch=64):

    batch_data_list = []
    batch_label_list = []
    batch_length = []

    for i in range(0, len(data_list), batch):
        upper = min(len(data_list), i + batch)
        data = pad_sequence(data_list[i:upper], batch_first=True, padding_value=0)
        label = pad_sequence(label_list[i:upper], batch_first=True, padding_value=0)
        batch_data_list.append(data)
        batch_label_list.append(label)
        batch_length.append(torch.tensor(length[i:upper]))
    return batch_data_list, batch_label_list, batch_length



In [9]:

batch_train_data, batch_train_label, batch_train_length = padding(train_data, train_label, train_length)
batch_valid_data, batch_valid_label, batch_valid_length = padding(valid_data, valid_label, valid_length)

batch_train_data[-1].shape

train_loader = list(zip(batch_train_data, batch_train_label, batch_train_length))
valid_loader = list(zip(batch_valid_data, batch_valid_label, batch_valid_length))



In [10]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
from torch import nn

In [33]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class LSTMTagger(nn.Module):

    def __init__(self, hidden_dim, tagset_size, input_dim=6):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        # LSTM層
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)

        # 線性層
        self.linear = nn.Linear(hidden_dim, tagset_size)
        self.relu = nn.ReLU()

    def init_hidden(self, batch_size):
        # 初始化隱藏狀態和細胞狀態
        return (torch.zeros(1, batch_size, self.hidden_dim),
                torch.zeros(1, batch_size, self.hidden_dim))

    def forward(self, sentence, hidden, lengths):
        """
        sentence: Tensor, shape (batch_size, seq_len, input_dim)
        lengths: List of sequence lengths (before padding)
        hidden: Initial hidden state

        Returns:
            tag_space: Tensor, shape (batch_size, seq_len, tagset_size)
            hidden: Final hidden state
        """
        # 動態打包序列
        packed_input = pack_padded_sequence(sentence, lengths, batch_first=True, enforce_sorted=False)

        # LSTM層
        packed_output, hidden = self.lstm(packed_input, hidden)

        # 解包序列
        lstm_out, _ = pad_packed_sequence(packed_output, batch_first=True)
        # lstm_out, hidden = self.lstm(sentence, hidden)
        # 線性層和激活函數
        tag_space = self.relu(self.linear(self.relu(lstm_out)))

        return tag_space, hidden


In [44]:
import torch
import torch.optim as optim
import torch.nn as nn

# 設置 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 定義訓練函數
def train_model(model, train_loader, valid_loader, train_length, valid_length, num_epochs=10, learning_rate=0.001):
    # 將模型移到 GPU
    model = model.to(device, dtype=torch.float32)
    # 使用 Adam 優化器
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    # 定義損失函數
    criterion = nn.SmoothL1Loss()
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0

        for inputs, labels, length in train_loader:
            # 將輸入和標籤移到 GPU
            inputs, labels, length = inputs.to(device, dtype=torch.float32), labels.to(device, dtype=torch.float32), length.to('cpu', dtype=torch.int64)
            
            # 初始化隱藏狀態
            hidden = model.init_hidden(batch_size=inputs.size(0))
            hidden = tuple([h.to(device, dtype=torch.float32) for h in hidden])

            

            # 清零梯度
            optimizer.zero_grad()
            # 前向傳播
            outputs, _ = model(inputs, hidden, length)
            # 計算損失
            loss = criterion(outputs.squeeze(), labels)
            # 反向傳播
            loss.backward()
            # 更新參數
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {total_loss / train_length:.4f}")

        # 驗證模型
        validate_model(model, valid_loader, criterion, valid_length)

# 定義驗證函數
def validate_model(model, valid_loader, criterion, valid_length):
    model.eval()
    total_loss = 0.0
    error = 0
    with torch.no_grad():
        for inputs, labels, length in valid_loader:
            # 將輸入和標籤移到 GPU
            inputs, labels, length = inputs.to(device, dtype=torch.float32), labels.to(device, dtype=torch.float32), length.to('cpu', dtype=torch.int64)
            
            # 初始化隱藏狀態
            hidden = model.init_hidden(batch_size=inputs.size(0))
            hidden = tuple([h.to(device, dtype=torch.float32) for h in hidden])

            # 前向傳播
            outputs, _ = model(inputs, hidden, length)
            # 計算損失
            loss = criterion(outputs.squeeze(), labels.squeeze())
            error += abs(outputs.view(-1) - labels.view(-1)).sum() / inputs.shape[0] / inputs.shape[1]
            total_loss += loss.item()

    print(f"Validation Loss: {total_loss / valid_length:.4f}, valid error: {error / valid_length}")
    return total_loss


In [45]:
model = LSTMTagger(256, 1, input_dim=5)
train_length, valid_length = len(batch_train_data), len(batch_valid_data)
train_model(model, train_loader, valid_loader, train_length, valid_length, 500)


Epoch [1/500], Training Loss: 209.0888
Validation Loss: 151.0277, valid error: 151.3748016357422
Epoch [2/500], Training Loss: 207.4958
Validation Loss: 150.4292, valid error: 150.77407836914062
Epoch [3/500], Training Loss: 206.8555
Validation Loss: 150.1667, valid error: 150.52328491210938
Epoch [4/500], Training Loss: 206.3188
Validation Loss: 150.0091, valid error: 150.36508178710938
Epoch [5/500], Training Loss: 205.9515
Validation Loss: 149.9818, valid error: 150.34471130371094
Epoch [6/500], Training Loss: 205.4910
Validation Loss: 149.7435, valid error: 150.1033477783203
Epoch [7/500], Training Loss: 205.2710
Validation Loss: 148.7288, valid error: 149.09402465820312
Epoch [8/500], Training Loss: 204.6983
Validation Loss: 148.6452, valid error: 149.0135040283203
Epoch [9/500], Training Loss: 202.4234
Validation Loss: 145.9071, valid error: 146.27822875976562
Epoch [10/500], Training Loss: 199.7456
Validation Loss: 144.6718, valid error: 145.03475952148438
Epoch [11/500], Traini

In [53]:
def show(model, loader, device):
    model.eval()
    error = 0
    data_num = 0
    with torch.no_grad():
        for inputs, labels, lengths in loader:
            inputs, lengths = inputs.to(device, dtype=torch.float32), lengths.to('cpu', dtype=torch.int64)
            
            hidden = model.init_hidden(batch_size=inputs.size(0))
            hidden = tuple([h.to(device, dtype=torch.float32) for h in hidden])
            outputs, _ = model(inputs, hidden, lengths)
            outputs = outputs.to('cpu', dtype=torch.float32).view(labels.shape[0], -1)

            
            for i in range(labels.shape[0]):
                length = lengths[i]
                error += abs(outputs[i, :length] - labels[i, :length]).sum()
                data_num += length

    print(float(error) / int(data_num), int(data_num))

np.set_printoptions(precision=2, suppress=True)
print("train MAE")
show(model, train_loader, device)
print("valid MAE")
show(model, valid_loader, device)

train MAE
13.43122487478052 125543
valid MAE
24.77413183874909 6147
